# Derivation of DCM for Calcium Imaging (Based on Jung.et.al 2019)

### Latent variable model

The typical place to start is the neuronal state equation
$$ \vec{x}(t+1) = A\vec{x}(t) + B \vec{x}(t) \vec{u}(t) + C\vec{u}(t) + \nu, \;\;\;\;\; \nu \sim \mathcal{N}(\vec{\mu}, \Sigma)$$

However, Jung2019 start with a convolution model of $Ca^{2+}$ potential $V$ instead

$$V_i(t) = h_{i}(t) \otimes (Inp_i(t) + \Sigma_j \gamma_{ij} \sigma_j(V_i - T_i))$$

where $i$ denotes the index of a neuronal population, $h_i(t) = H_i k_i t e^{-k_i t}$ is the response function of a population to incoming current, $Inp_i(t)$ is the external input to each neuron, $\gamma_{ij}$ are the connection weights, $\sigma_i(V_i - T_i) = \frac{f_{max}}{1 - e^{R(V_i - T_i)}}$ is the nonlinear activation function and $T_i$ is the firing threshold.

Doubly-differentiating the model results in a 2nd order ODE, which can be split into two first-order ODE's by introducing the $Ca^{2+}$ current $I$

$$
\begin{eqnarray}
\dot{V}_i &=& I_i \\
\dot{I}_i &=& k_i H_i (Inp_i + \Sigma_j \gamma_{ij} \sigma_j(V_i - T_i))) - 2k_i \dot{V}_i - k_i^2 (V_i - T_i)
\end{eqnarray}
$$

Thus the values of $V(t)$ and $I(t)$ are deterministically determined by their own values a short time beforehand plus the external inputs $Inp(t)$

### Forwards model

It is well-known that $Ca^{2+}$ indicators such as GCaMP6f used in this study work as integrators of the $Ca^{2+}$ concentration with a timescale of $\tau_{ca} \sim 0.5s$. Jung2019 present the following model, relating the potential $V$, calcium concentration $[Ca^{2+}]$ and the observable fluorescence signal $F$

$$\frac{d}{dt}[Ca^{2+}] = K_{ca} g_{ca} \sigma(V_i - T_i) - \frac{[Ca^{2+}] - [Ca^{2+}]_{base}}{\tau_{ca}}$$

$$F = d_F + k_F \frac{[Ca^{2+}]}{[Ca^{2+}] + k_d}$$

### Observer noise model

While not explicitly mentioned in Jung2019, the simplest inversion problem one can solve is observer noise. in particular, the measured fluorescence is given by $$F_m(t) \sim \mathcal{N}(F(t), \Sigma)$$ given some constant normal noise covariance. In this case, the likelihood function, the likelihood function $P[F_m(t) | \theta(t)]$, where $\theta(t) = \{V(t), I(t), [Ca^{2+}](t), \gamma_{ij}, \tau, ...\}$ is given by

$$P[F_m(t) | \theta(t)] = Gau(F_m(t) - F(t, \theta(t)), \Sigma)$$

And the total likelihood of the data 

$$L[F_m | \theta] = \prod_t Gau(F_m(t) - F(t, \theta(t)), \Sigma)$$

Now, since the $\theta(t)$ are not known, it would make sense to eliminate them from the model. Using the fact that the model is deterministic, we can replace $F$ with an equivalent function $\tilde{F}$, which only requires initial values of parameters to compute the estimate of the observed signal
$$ F(t, \theta(t) = \tilde{F}(t, \theta(0))$$
$\tilde{F}$ is defined by the following algorithm
* Initialize ODE at time $t=0$ using provided values of $\theta_0$
* Forwards-integrate ODE to get values of $\theta(t_i)$ for desired time points $\{t_i\}$
* Evaluate $\tilde{F} = F(t_i, \theta(t_i))$ using the forwards model

### Priors


### Model inversion
